In [1]:
import cv2
import numpy as np
import os
 
# read the image
image = cv2.imread('./photos_adrien/3.jpg')#avec contour.jpg
cv2.imshow('Uploadedimage', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
image_inverted = np.invert(image)


In [2]:
# convert the image to grayscale format
img_gray = cv2.cvtColor(image_inverted, cv2.COLOR_BGR2GRAY)
cv2.imshow('Binary inverted image', img_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
# apply binary thresholding
ret, thresh = cv2.threshold(img_gray, 100, 255, cv2.THRESH_BINARY)
# visualize the binary image
cv2.imshow('Binary inverted thresholded image', thresh)
cv2.waitKey(0)
cv2.imwrite('image_thres1.jpg', thresh)
cv2.destroyAllWindows()

In [4]:
def card_or_not(thresh,maxnbcard):  
    CARD_MAX_AREA = 2000000
    CARD_MIN_AREA = 100000
    MAX_CARD_NB = maxnbcard
    # Find contours and sort their indices by contour size
    cnts,hier = cv2.findContours(image=thresh,mode=cv2.RETR_TREE,method=cv2.CHAIN_APPROX_SIMPLE)
    index_sort = sorted(range(len(cnts)), key=lambda i : cv2.contourArea(cnts[i]),reverse=True)


    # Otherwise, initialize empty sorted contour and hierarchy lists
    cnts_sort = []
    hier_sort = []
    cnt_is_card = []

    # Fill empty lists with sorted contour and sorted hierarchy. Now,
    # the indices of the contour list still correspond with those of
    # the hierarchy list. The hierarchy array can be used to check if
    # the contours have parents or not.
    for i in index_sort:
        cnts_sort.append(cnts[i])
        hier_sort.append(hier[0][i])

    # Determine which of the contours are cards by applying the
    # following criteria: 1) Smaller area than the maximum card size,
    # 2), bigger area than the minimum card size, 3) have no parents,
    # and 4) have four corners
    size_list=[]
    contourlist=[]

    #determining avg contour size
    for i in range(len(cnts_sort)):
        contourlist.append(cv2.contourArea(cnts_sort[i]))
    size_avg=np.sum(contourlist)/MAX_CARD_NB

    print('avg_size:',size_avg)


    for i in range(len(cnts_sort)):

        size_list.append((i,cv2.contourArea(cnts_sort[i])))
        size = cv2.contourArea(cnts_sort[i])
        peri = cv2.arcLength(cnts_sort[i],True)
        approx = cv2.approxPolyDP(cnts_sort[i],0.01*peri,True)

            
        if ((size > 0.7*size_avg) and (size < MAX_CARD_NB*size_avg) #(size < CARD_MAX_AREA) and (size > CARD_MIN_AREA) avant
            and (hier_sort[i][3] == -1 or hier_sort[i][2]==-1) and (len(approx) == 4)):

            cnt_is_card.append(i)
    return [cnts_sort[i] for i in cnt_is_card]



In [5]:
contours=card_or_not(thresh,10)  
print(contours)

avg_size: 123124.95
[array([[[327, 585]],

       [[325, 587]],

       [[324, 587]],

       ...,

       [[339, 586]],

       [[335, 586]],

       [[334, 585]]], dtype=int32)]


In [6]:
contoured_img=cv2.drawContours(image.copy(),contours,-1,(0, 255, 0),3)
# see the results
cv2.imshow('card contours', contoured_img)
cv2.waitKey(0)
cv2.imwrite('cardcontour.jpg', contoured_img)
cv2.destroyAllWindows()

In [7]:
def flattener(image, pts, w, h):
    """Flattens an image of a card into a top-down 200x300 perspective.
    Returns the flattened, re-sized, grayed image.
    See www.pyimagesearch.com/2014/08/25/4-point-opencv-getperspective-transform-example/"""
    temp_rect = np.zeros((4,2), dtype = "float32")
    
    s = np.sum(pts, axis = 2)

    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]

    diff = np.diff(pts, axis = -1)
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]

    # Need to create an array listing points in order of
    # [top left, top right, bottom right, bottom left]
    # before doing the perspective transform

    if w <= 0.8*h: # If card is vertically oriented
        temp_rect[0] = tl
        temp_rect[1] = tr
        temp_rect[2] = br
        temp_rect[3] = bl

    if w >= 1.2*h: # If card is horizontally oriented
        temp_rect[0] = bl
        temp_rect[1] = tl
        temp_rect[2] = tr
        temp_rect[3] = br

    # If the card is 'diamond' oriented, a different algorithm
    # has to be used to identify which point is top left, top right
    # bottom left, and bottom right.
    
    if w > 0.8*h and w < 1.2*h: #If card is diamond oriented
        # If furthest left point is higher than furthest right point,
        # card is tilted to the left.
        if pts[1][0][1] <= pts[3][0][1]:
            # If card is titled to the left, approxPolyDP returns points
            # in this order: top right, top left, bottom left, bottom right
            temp_rect[0] = pts[1][0] # Top left
            temp_rect[1] = pts[0][0] # Top right
            temp_rect[2] = pts[3][0] # Bottom right
            temp_rect[3] = pts[2][0] # Bottom left

        # If furthest left point is lower than furthest right point,
        # card is tilted to the right
        if pts[1][0][1] > pts[3][0][1]:
            # If card is titled to the right, approxPolyDP returns points
            # in this order: top left, bottom left, bottom right, top right
            temp_rect[0] = pts[0][0] # Top left
            temp_rect[1] = pts[3][0] # Top right
            temp_rect[2] = pts[2][0] # Bottom right
            temp_rect[3] = pts[1][0] # Bottom left
            
        
    maxWidth = 600
    maxHeight = 1000

    # Create destination array, calculate perspective transform matrix,
    # and warp card image
    dst = np.array([[0,0],[maxWidth-1,0],[maxWidth-1,maxHeight-1],[0, maxHeight-1]], np.float32)
    M = cv2.getPerspectiveTransform(temp_rect,dst)
    warp = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    #warp = cv2.cvtColor(warp,cv2.COLOR_BGR2GRAY)

        

    return warp

In [8]:
def contours_detailer(contour):
    # Find perimeter of card and use it to approximate corner points
    peri = cv2.arcLength(contour,True)
    approx = cv2.approxPolyDP(contour,0.01*peri,True)
    pts = np.float32(approx)
    points = pts

    # Find width and height of card's bounding rectangle
    x,y,w,h = cv2.boundingRect(contour)
    width, height = w, h

    # Find center point of card by taking x and y average of the four corners.
    average = np.sum(pts, axis=0)/len(pts)
    cent_x = int(average[0][0])
    cent_y = int(average[0][1])
    center = [cent_x, cent_y]

    return pts,w,h,center

iter=8
for i,contour in enumerate(contours):
    pts,w,h,center=contours_detailer(contour)
    # Warp card into 600x1000 flattened image using perspective transform
    name='crop_'+str(i)+'_'+str(iter)+'.jpg'
    path='./photos_adrien/'
    finalpath=os.path.join(path,name)
    print(finalpath)
    warp = flattener(image, pts, w, h)
    print(type(warp),warp.shape)
    cv2.imshow('None approximation', warp)
    cv2.waitKey(0)
    cv2.imwrite(finalpath, warp)
    cv2.destroyAllWindows()

./photos_adrien/crop_0_8.jpg
<class 'numpy.ndarray'> (1000, 600, 3)


In [1]:
!python3 -m pip install torch torchvision torchaudio

  Using cached torch-1.13.1-cp39-cp39-manylinux1_x86_64.whl (887.4 MB)
  Using cached torchvision-0.14.1-cp39-cp39-manylinux1_x86_64.whl (24.2 MB)
  Using cached torchaudio-0.13.1-cp39-cp39-manylinux1_x86_64.whl (4.2 MB)


In [2]:
!pip list --format=freeze

absl-py==1.4.0
aiohttp==3.8.1
aiosignal==1.2.0
alabaster==0.7.12
alembic==1.8.1
altair==4.2.0
ansys-corba==0.1.0
anyio==3.6.2
appdirs==1.4.4
argon2-cffi==21.1.0
arrow==1.2.3
asgiref==3.5.2
astunparse==1.6.3
async-timeout==4.0.2
attr==0.3.1
attrs==21.2.0
autobahn==22.7.1
Automat==22.10.0
autoviz==0.1.58
azure-common==1.1.28
azure-core==1.26.0
azure-identity==1.12.0
azure-mgmt-core==1.3.2
azure-mgmt-rdbms==10.1.0
azure-mgmt-resource==21.2.1
azure-mgmt-storage==20.1.0
azure-storage-blob==12.14.1
azure-storage-file-datalake==12.9.1
Babel==2.11.0
backcall==0.2.0
beautifulsoup4==4.11.1
bleach==4.1.0
blinker==1.4
blis==0.7.9
bokeh==2.4.3
boto==2.49.0
boto3==1.16.63
botocore==1.19.63
boxing==0.1.4
branca==0.5.0
Brotli==1.0.9
cachetools==5.2.0
cassandra-pylib==0.0.0
catalogue==1.0.2
certifi==2021.10.8
cffi==1.14.6
channels==4.0.0
charset-normalizer==2.0.12
click==8.1.3
cloudpickle==2.2.0
colorama==0.4.6
colorcet==3.0.1
commonmark==0.9.1
conda==4.3.16
ConfigArgParse==1.5.3
constantly==15.1.0
con

In [ ]:
import torch
import os

os.environ["USE_TORCH"] = "1"

In [ ]:
from doctr.io import DocumentFile
from doctr.models import ocr_predictor, sar_resnet31,crnn_mobilenet_v3_small,db_resnet50

In [ ]:
doc = DocumentFile.from_images("crop_0_4.jpg")


rec_model = crnn_mobilenet_v3_small(pretrained=False)
det_model=db_resnet50(pretrained=False)
rec_model.load_state_dict(torch.load("/content/crnn_mobilenet_v3_small_pretranied.pt", map_location="cpu"))
det_model.load_state_dict(torch.load("/content/db_resnet50_latest.pt", map_location="cpu"))
predictor = ocr_predictor(reco_arch=rec_model, 
                          det_arch=det_model,
                          pretrained_backbone=True,
                          assume_straight_pages=True,
                          preserve_aspect_ratio=True,
                          symmetric_pad=True).cuda()

res = predictor(doc)
res.show(doc)